In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import os
import cmasher as cmr
from matplotlib.colors import Normalize
import scipy.special as sc

In [ ]:
plt.style.use("ncr-paper.mplstyle")

In [ ]:
# "qdset" is a xarray.Dataset storing the (16, 50, 84) percentile values and mean and std for each model's time series into an effectively 3D array.
with xr.open_dataarray("percentiles_all.nc") as qdset:
    qmean = qdset.sel(q="mean")

zp_basedir = "lowZ-zprof-data"
newzp = dict()
for m in qdset["name"].data:
    f = os.path.join(zp_basedir, f"{m}_newzp.nc")
    with xr.open_dataset(f) as dset:
        newzp[m] = dset

In [ ]:
for k in newzp:
    print(k, newzp[k].time_code[0].data)

In [ ]:
def retrieve_timeseries(m):
    outdir = "./lowZ-coolheat-data"
    outfile = os.path.join(outdir, f"{m}_PEheating.nc")
    outfile2 = os.path.join(outdir, f"{m}_phase_vmeans.nc")
    outfile3 = os.path.join(outdir, f"{m}_phase_nmeans.nc")

    with xr.open_dataarray(outfile) as da:
        da.load()
    with xr.open_dataset(outfile2) as ds1:
        ds1.load()
    with xr.open_dataset(outfile3) as ds2:
        ds2.load()

    return da, ds1, ds2

In [ ]:
def get_color(Zdust, cmap=cmr.guppy, Zmin=-1.3, Zmax=0.3):
    norm = Normalize(vmin=Zmin, vmax=Zmax)
    c = cmap(norm(np.log10(Zdust)))
    return c


def get_kwargs(name, cmap=cmr.guppy, Zmin=-1.3, Zmax=0.3):
    if "Zd0.025" in name:
        marker = "s"
    elif "Om01" in name:
        marker = "v"
    elif "Om02" in name:
        marker = "^"
    elif "b10" in name and "S05" not in name:
        marker = "*"
    else:
        marker = "o"
    if "Zd0.025" in name:
        Zdust = 0.025
    elif "Zd0.1" in name:
        Zdust = 0.1
    elif "Zd0.3" in name:
        Zdust = 0.3
    elif "Zd1" in name:
        Zdust = 1
    c = get_color(Zdust)
    return Zdust, dict(marker=marker, color=c)

In [ ]:
def plot_points(x, y, **kwargs):
    xlow, xmid, xhigh = np.nanquantile(x, [0.16, 0.50, 0.84])
    ylow, ymid, yhigh = np.nanquantile(y, [0.16, 0.50, 0.84])
    qx = np.array([xlow, xmid, xhigh])
    qy = np.array([ylow, ymid, yhigh])
    # print(qx,qy)
    xerr = np.array([qx[1] - qx[0], qx[2] - qx[1]]).T
    yerr = np.array([qy[1] - qy[0], qy[2] - qy[1]]).T

    plt.errorbar(
        qx[1],
        qy[1],
        xerr=xerr.reshape(2, 1),
        yerr=yerr.reshape(2, 1),
        markersize=5,
        elinewidth=1,
        ecolor="k",
        markeredgecolor="k",
        ls="",
        **kwargs,
    )

    sca = plt.scatter(x, y, s=10, alpha=0.1, linewidths=0, **kwargs)
    plt.yscale("log")
    plt.xscale("log")

In [ ]:
outdir = "./lowZ-coolheat-figures"
os.makedirs(outdir, exist_ok=True)

In [ ]:
for m in qdset["name"].data:
    da, vavg, navg = retrieve_timeseries(m)
    Zdust, kwargs = get_kwargs(m)
    ds = da.to_dataset("variable")
    ds["Zdust"] = Zdust * ds.time / ds.time
    plot_points(ds["tau"], ds["ftau"], **kwargs)

tau = np.logspace(-3, 2)
ftau = (1 - sc.expn(2, tau / 2)) / tau
plt.plot(tau, ftau, "k")

plt.ylabel(
    r"$f_{\tau}\equiv 4\pi \langle J_{\rm FUV}\rangle_{\rm 2p}/\dot{S}_{\rm FUV}$"
)
plt.xlabel(r"$\tau = Z_d^\prime \sigma_{\rm FUV} N_H$")
plt.yscale("log")
plt.ylim(1.0e-1, 10)
plt.xlim(1.0e-3, 20)
plt.xscale("log")

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(12, 10))
for m in qdset["name"].data:
    da, vavg, navg = retrieve_timeseries(m)
    Zdust, kwargs = get_kwargs(m)
    navg["ratio"] = navg["heat_rate"] / navg["cool_rate"]
    vavg["ratio"] = vavg["heat_rate"] / vavg["cool_rate"]

    for i, ph in enumerate(["CNM", "Cold", "WNM"]):
        plt.sca(axes[0, i])
        d = vavg.sel(phase=ph)

        plot_points(d["tcool"], d["ratio"], **kwargs)
        plt.xlabel(r"$\langle t_{\rm cool}\rangle_V \,[{\rm Myr}]$")
        plt.title(ph)
        plt.sca(axes[1, i])
        plot_points(d["theat"], d["ratio"], **kwargs)
        plt.xlabel(r"$\langle t_{\rm heat}\rangle_V \,[{\rm Myr}]$")
for ax in axes.flatten():
    plt.sca(ax)
    plt.xlim(1.0e-3, 1.0e2)
    plt.yscale("linear")
    plt.ylim(0, 1.2)
    plt.xscale("log")
    plt.ylabel(r"$\langle\mathcal{G}\rangle_V/\langle\mathcal{L}\rangle_V$")
plt.tight_layout()
plt.savefig(os.path.join(outdir, "thermal_equilibrium.png"), bbox_inches="tight")

In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(15, 8))
for m in qdset["name"].data:
    da, vavg, navg = retrieve_timeseries(m)
    zp = newzp[m]
    Zdust, kwargs = get_kwargs(m)

    for ph, ax in zip(["CNM", "Cold", "WNM", "WHIM"], axes.T):
        if ph == "Cold":
            zpph = zp.sel(phase=["CNM", "UNM"]).sum(dim="phase")
        else:
            zpph = zp.sel(phase=ph)
        vz = np.sqrt((2 * zpph["Ek3"].sum(dim="z")) / zpph["d"].sum(dim="z"))
        H = np.sqrt((zpph["d"] * zpph["z"] ** 2).sum(dim="z") / zpph["d"].sum(dim="z"))
        tver = H / vz

        zpph = zpph.sel(z=slice(-50, 50))
        tcool = 1.5 * zpph["P"].sum(dim="z") / zpph["cool"].sum(dim="z")
        x = (zpph["Ek1"] + zpph["dEk2"] + zpph["Ek3"]).sum(dim="z") / tver
        y = (zpph["cool"] - zpph["heat"]).sum(dim="z")

        plt.sca(ax[0])
        plot_points(x, y, **kwargs)
        plt.xlabel(r"$(\rho v^2/2)/t_{\rm ver}$")
        x = (zpph["Ek1"] + zpph["dEk2"] + zpph["Ek3"]).sum(dim="z") / tcool

        plt.sca(ax[1])
        plot_points(x, y, **kwargs)
        plt.xlabel(r"$(\rho v^2/2)/t_{\rm cool}$")

for ph, ax in zip(["CNM", "Cold", "WNM", "WHIM"] * 2, axes.flatten()):
    plt.sca(ax)
    plt.xscale("log")
    plt.yscale("log")
    plt.title(ph)
    plt.plot([1.0e2, 1.0e12], [1.0e2, 1.0e12], color="k", ls="--")
    plt.xlim(1.0e2, 1.0e12)
    plt.ylim(1.0e2, 1.0e12)
    plt.gca().set_aspect("equal")

    plt.ylabel(r"$\mathcal{G}_{\rm diss} \equiv \mathcal{L}-\mathcal{G}$")
plt.tight_layout()
plt.savefig(os.path.join(outdir, "Gdiss_midplane.png"), bbox_inches="tight")

In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(15, 8))
for m in qdset["name"].data:
    da, vavg, navg = retrieve_timeseries(m)
    zp = newzp[m]
    Zdust, kwargs = get_kwargs(m)

    for ph, ax in zip(["CNM", "Cold", "WNM", "WHIM"], axes.T):
        if ph == "Cold":
            zpph = zp.sel(phase=["CNM", "UNM"]).sum(dim="phase")
        else:
            zpph = zp.sel(phase=ph)
        vz = np.sqrt((2 * zpph["Ek3"].sum(dim="z")) / zpph["d"].sum(dim="z"))
        H = np.sqrt((zpph["d"] * zpph["z"] ** 2).sum(dim="z") / zpph["d"].sum(dim="z"))
        tver = H / vz

        # zpph = zpph.sel(z=slice(-50,50))
        tcool = 1.5 * zpph["P"].sum(dim="z") / zpph["cool"].sum(dim="z")
        x = (zpph["Ek1"] + zpph["dEk2"] + zpph["Ek3"]).sum(dim="z") / tver
        y = (zpph["cool"] - zpph["heat"]).sum(dim="z")

        plt.sca(ax[0])
        plot_points(x, y, **kwargs)
        plt.xlabel(r"$(\rho v^2/2)/t_{\rm ver}$")
        x = (zpph["Ek1"] + zpph["dEk2"] + zpph["Ek3"]).sum(dim="z") / tcool

        plt.sca(ax[1])
        plot_points(x, y, **kwargs)
        plt.xlabel(r"$(\rho v^2/2)/t_{\rm cool}$")

for ph, ax in zip(["CNM", "Cold", "WNM", "WHIM"] * 2, axes.flatten()):
    plt.sca(ax)
    plt.xscale("log")
    plt.yscale("log")
    plt.title(ph)
    plt.plot([1.0e2, 1.0e12], [1.0e2, 1.0e12], color="k", ls="--")
    plt.xlim(1.0e2, 1.0e12)
    plt.ylim(1.0e2, 1.0e12)
    plt.gca().set_aspect("equal")

    plt.ylabel(r"$\mathcal{G}_{\rm diss} \equiv \mathcal{L}-\mathcal{G}$")
plt.tight_layout()
plt.savefig(os.path.join(outdir, "Gdiss_fullbox.png"), bbox_inches="tight")

In [ ]:
def get_integrated(zp):
    data = dict()
    # total volume occupied by each phase
    data["volume"] = zp["A"].sum(dim="z")
    # total cooling rate per volume in each phase
    data["netcool"] = zp["net_cool"].sum(dim="z")
    # total cooling rate per volume in each phase
    data["cooling"] = zp["cool"].sum(dim="z")
    # total heating rate in each phase
    data["heating"] = zp["heat"].sum(dim="z")
    # total kinethc energy dissipation rate in each phase
    data["mass"] = zp["d"].sum(dim="z")
    data["Ek"] = (zp["Ek1"] + zp["Ek2"] + zp["Ek3"]).sum(dim="z")
    data["dEk"] = (zp["Ek1"] + zp["dEk2"] + zp["Ek3"]).sum(dim="z")
    # scale height
    data["rhoz2"] = (zp["d"] * zp.z**2).sum(dim="z")
    # velocity dispersion
    data["Ek3"] = zp["Ek3"].sum(dim="z")
    # thermal energy
    data["Eth"] = 1.5 * zp["P"].sum(dim="z")

    warm = dict()
    cold = dict()
    unst = dict()
    whot = dict()
    hot = dict()
    for f in data:
        warm[f] = data[f].sel(phase=["WNM", "WIM"]).sum(dim="phase")
        cold[f] = data[f].sel(phase=["CMM", "CNM"]).sum(dim="phase")
        unst[f] = data[f].sel(phase=["UNM"]).sum(dim="phase")
        whot[f] = data[f].sel(phase=["WHIM"]).sum(dim="phase")
        hot[f] = data[f].sel(phase=["HIM"]).sum(dim="phase")

    # derived
    for data in [cold, unst, warm, whot, hot]:
        H = np.sqrt(data["rhoz2"] / data["mass"])
        vz = np.sqrt(2.0 * data["Ek3"] / data["mass"])
        data["tver"] = H / vz
        data["Edot_diss"] = data["Ek"] / data["tver"]
        data["tcool"] = data["Eth"] / data["netcool"]
    return cold, unst, warm, whot, hot

# Cooling and Heating

The net cooling in each phase is
$$ L_{\rm net}^{\rm ph} \equiv L^{\rm ph} - G^{\rm ph} - \dot{E}_{\rm diss}^{\rm ph} - L_{\rm net}^{\rm ph+1} $$

In the coldest phase, $L_{\rm net}^{\rm ph} = 0$ should satisfy, i.e., no energy transferred to the other phase.

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(15, 4), sharex="col")
# ax1, ax2 = axes
for m in qdset["name"].data:
    da, vavg, navg = retrieve_timeseries(m)
    zp = newzp[m]
    Zdust, kwargs = get_kwargs(m)

    cold, unst, warm, whot, hot = get_integrated(zp)

    for ax, d, dp1 in zip(axes, [whot, warm, unst, cold], [hot, whot, warm, unst]):
        H = np.sqrt(d["rhoz2"] / d["mass"])
        vz = np.sqrt(2.0 * d["Ek3"] / d["mass"])
        tver = H / vz
        Edot = d["Ek"] / tver
        d["tcool"] = d["Eth"] / d["netcool"]
        if "tcool" not in dp1:
            dp1["tcool"] = dp1["Eth"] / dp1["netcool"]
        d["Edot_diss"] = Edot
        d["net_cooling"] = d["cooling"] - d["heating"] - d["Edot_diss"]
        if "net_cooling" in dp1:
            d["net_cooling"] -= dp1["net_cooling"]
        plt.sca(ax)

    for ax, d, dp1 in zip(axes, [whot, warm, unst, cold], [hot, whot, warm, unst]):
        plt.sca(ax)
        plot_points(d["tcool"], dp1["tcool"], **kwargs)
        plt.plot([1.0e-12, 1.0e15], [1.0e-12, 1.0e15], color="k", ls="--")
        plt.xlim(1.0e-3, 1.0e3)
        plt.ylim(1.0e-3, 1.0e3)

for ax, d, dp1 in zip(axes, ["wh", "w", "u", "c"], ["h", "wh", "w", "u"]):
    plt.sca(ax)
    plt.xlabel(f"$t_{{\\rm cool}}^{{\\rm {d}}}$")
    plt.ylabel(f"$t_{{\\rm cool}}^{{\\rm {dp1}}}$")

plt.tight_layout()
plt.savefig(os.path.join(outdir, "tcool_comparison.png"), bbox_inches="tight")

In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(18, 8), sharex="col")
ax1, ax2 = axes
for m in qdset["name"].data:
    da, vavg, navg = retrieve_timeseries(m)
    zp = newzp[m]
    Zdust, kwargs = get_kwargs(m)

    cold, unst, warm, whot, hot = get_integrated(zp)

    for ax1_, ax2_, d, ph in zip(
        ax1, ax2, [whot, warm, unst, cold], ["wh", "w", "u", "c"]
    ):
        y = d["cooling"] - d["heating"]
        x = d["cooling"]
        plt.sca(ax1_)
        plot_points(x, y, **kwargs)
        plt.ylabel(f"$L^{{\\rm {ph}}} - G^{{\\rm {ph}}}$")
        plt.sca(ax2_)
        plot_points(x, y / x, **kwargs)
        plt.ylabel(f"$(L^{{\\rm {ph}}} - G^{{\\rm {ph}}})/L^{{\\rm {ph}}}$")
        plt.xlabel(f"$L^{{\\rm {ph}}}$")

for ax in ax1:
    plt.sca(ax)
    plt.plot([1.0e2, 1.0e15], [1.0e2, 1.0e15], color="k", ls="--")
    plt.xlim(1.0e3, 1.0e13)
    plt.ylim(1.0e3, 1.0e13)


for ax in ax2:
    plt.sca(ax)
    plt.axhline(1, color="k", ls="--")
    plt.ylim(0.01, 10)
plt.tight_layout()
plt.savefig("./lowZ-coolheat-figures/netcool_cool.png")

In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(18, 8), sharex="col")
ax1, ax2 = axes
for m in qdset["name"].data:
    da, vavg, navg = retrieve_timeseries(m)
    zp = newzp[m]
    Zdust, kwargs = get_kwargs(m)

    cold, unst, warm, whot, hot = get_integrated(zp)

    for ax1_, ax2_, d, ph in zip(
        ax1, ax2, [whot, warm, unst, cold], ["wh", "w", "u", "c"]
    ):
        y = d["cooling"] - d["heating"]
        x = d["Edot_diss"]
        plt.sca(ax1_)
        plot_points(x, y, **kwargs)
        plt.ylabel(f"$L^{{\\rm {ph}}} - G^{{\\rm {ph}}}$")
        plt.sca(ax2_)
        plot_points(x, y / x, **kwargs)
        plt.ylabel(
            f"$(L^{{\\rm {ph}}} - G^{{\\rm {ph}}})/\\dot{{E}}_{{\\rm diss}}^{{\\rm {ph}}}$"
        )
        plt.xlabel(f"$\\dot{{E}}_{{\\rm diss}}^{{\\rm {ph}}}$")

for ax in ax1:
    plt.sca(ax)
    plt.plot([1.0e2, 1.0e15], [1.0e2, 1.0e15], color="k", ls="--")
    plt.xlim(1.0e3, 1.0e12)
    plt.ylim(1.0e3, 1.0e12)


for ax in ax2:
    plt.sca(ax)
    plt.axhline(1, color="k", ls="--")
    plt.ylim(0.1, 1000)
plt.tight_layout()
plt.savefig("./lowZ-coolheat-figures/netcool_edot_diss.png")

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
for m in qdset["name"].data:
    da, vavg, navg = retrieve_timeseries(m)
    zp = newzp[m]
    Zdust, kwargs = get_kwargs(m)

    cold, unst, warm, whot, hot = get_integrated(zp)

    d, dp1 = (warm, whot)
    x = dp1["cooling"]
    y = d["cooling"] - d["heating"]
    plt.sca(ax1)
    plot_points(x, y, **kwargs)
    plt.ylabel(r"$L^{\rm w} - G^{\rm w}$")
    plt.xlabel(r"$L^{\rm wh}$")
    plt.sca(ax2)
    plot_points(x, y / x, **kwargs)
    plt.ylabel(r"$(L^{\rm w} - G^{\rm w})/L_{\rm wh}$")
    plt.xlabel(r"$L^{\rm wh}$")

plt.sca(ax1)
plt.plot([1.0e2, 1.0e15], [1.0e2, 1.0e15], color="k", ls="--")
plt.xlim(1.0e5, 1.0e13)
plt.ylim(1.0e5, 1.0e13)

plt.sca(ax2)
plt.axhline(1, color="k", ls="--")
plt.xlim(1.0e5, 1.0e13)
plt.ylim(0.01, 100)
plt.tight_layout()
plt.savefig("./lowZ-coolheat-figures/netcool_warm_whot.png")

In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(18, 8), sharex="col")
ax1, ax2 = axes
for m in qdset["name"].data:
    da, vavg, navg = retrieve_timeseries(m)
    zp = newzp[m]
    Zdust, kwargs = get_kwargs(m)

    cold, unst, warm, whot, hot = get_integrated(zp)

    d, dp1 = (warm, whot)
    xlist = [
        d["cooling"],
        d["Edot_diss"],
        dp1["netcool"],
        #  dp1['netcool'],
        d["Edot_diss"] + dp1["netcool"],
    ]
    for ax1_, ax2_, x in zip(ax1, ax2, xlist):
        y = d["cooling"] - d["heating"]
        plt.sca(ax1_)
        plot_points(x, y, **kwargs)
        plt.ylabel(r"$L^{\rm w} - G^{\rm w}$")
        plt.sca(ax2_)
        plot_points(x, y / x, **kwargs)
        plt.ylabel(r"$(L^{\rm w} - G^{\rm w})/x$")

for ax in ax1:
    plt.sca(ax)
    plt.plot([1.0e2, 1.0e15], [1.0e2, 1.0e15], color="k", ls="--")
    plt.xlim(1.0e5, 1.0e13)
    plt.ylim(1.0e5, 1.0e13)


for ax, xlab in zip(
    ax2,
    [
        r"$L^{\rm w}$",
        r"$\dot{E}_{\rm diss}^{\rm w}$",
        r"$(L-G)^{\rm wh}$",
        # r"$L_{\rm net}^{\rm wh}$",
        r"$\dot{E}_{\rm diss}^{\rm w}+(L-G)^{\rm wh}$",
    ],
):
    plt.sca(ax)
    plt.axhline(1, color="k", ls="--")
    plt.xlabel(xlab)
    plt.ylim(0.01, 100)
plt.tight_layout()
plt.savefig("./lowZ-coolheat-figures/netcool_warm_whot_all.png")

In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(18, 8), sharex="col")
ax1, ax2 = axes
for m in qdset["name"].data:
    da, vavg, navg = retrieve_timeseries(m)
    zp = newzp[m]
    Zdust, kwargs = get_kwargs(m)

    cold, unst, warm, whot, hot = get_integrated(zp)

    d, dp1 = (warm, whot)
    xlist = [
        d["cooling"] / d["volume"],
        d["Edot_diss"] / d["volume"],
        dp1["netcool"] / dp1["volume"],
        #  dp1['netcool'],
        (d["Edot_diss"] + dp1["netcool"]) / (d["volume"] + dp1["volume"]),
    ]
    for ax1_, ax2_, x in zip(ax1, ax2, xlist):
        y = (d["cooling"] - d["heating"]) / d["volume"]
        plt.sca(ax1_)
        plot_points(x, y, **kwargs)
        plt.ylabel(r"${\cal L}^{\rm w} - {\cal G}^{\rm w}$")
        plt.sca(ax2_)
        plot_points(x, y / x, **kwargs)
        plt.ylabel(r"$({\cal L}^{\rm w} - {\cal G}^{\rm w})/x$")

for ax in ax1:
    plt.sca(ax)
    plt.plot([1.0e-5, 1.0e5], [1.0e-5, 1.0e5], color="k", ls="--")
    plt.xlim(1.0e-5, 1.0e5)
    plt.ylim(1.0e-5, 1.0e5)


for ax, xlab in zip(
    ax2,
    [
        r"${\cal L}^{\rm w}$",
        r"$\dot{e}_{\rm diss}^{\rm w}$",
        r"$({\cal L}-{\cal G})^{\rm wh}$",
        r"$\dot{e}_{\rm diss}^{\rm w}+({\cal L}-{\cal G})^{\rm wh}$",
    ],
):
    plt.sca(ax)
    plt.axhline(1, color="k", ls="--")
    plt.xlabel(xlab)
    plt.ylim(0.01, 100)
plt.tight_layout()
plt.savefig("./lowZ-coolheat-figures/netcool_volume_warm_whot.png")

In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(18, 8), sharex="col")
ax1, ax2 = axes
for m in qdset["name"].data:
    da, vavg, navg = retrieve_timeseries(m)
    zp = newzp[m]
    Zdust, kwargs = get_kwargs(m)

    cold, unst, warm, whot, hot = get_integrated(zp)

    d, dp1, dp2 = (unst, warm, whot)
    xlist = [
        d["cooling"],
        d["Edot_diss"],
        dp1["netcool"] - dp2["netcool"],
        #  dp1['netcool'],
        d["Edot_diss"] + dp1["netcool"] - dp2["netcool"],
    ]
    for ax1_, ax2_, x in zip(ax1, ax2, xlist):
        y = d["cooling"] - d["heating"]
        plt.sca(ax1_)
        plot_points(x, y, **kwargs)
        plt.ylabel(r"$L^{\rm u} - G^{\rm u}$")
        plt.sca(ax2_)
        plot_points(x, y / x, **kwargs)
        plt.ylabel(r"$(L^{\rm u} - G^{\rm u})/x$")

for ax in ax1:
    plt.sca(ax)
    plt.plot([1.0e2, 1.0e15], [1.0e2, 1.0e15], color="k", ls="--")
    plt.xlim(1.0e5, 1.0e13)
    plt.ylim(1.0e5, 1.0e13)


for ax, xlab in zip(
    ax2,
    [
        r"$L^{\rm u}$",
        r"$\dot{E}_{\rm diss}^{\rm u}$",
        r"$(L-G)^{\rm w}$",
        # r"$L_{\rm net}^{\rm wh}$",
        r"$\dot{E}_{\rm diss}^{\rm u}+(L-G)^{\rm w}$",
    ],
):
    plt.sca(ax)
    plt.axhline(1, color="k", ls="--")
    plt.xlabel(xlab)
    plt.ylim(0.01, 100)
plt.tight_layout()
plt.savefig("./lowZ-coolheat-figures/netcool_unst_warm.png")

In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(18, 8), sharex="col")
ax1, ax2 = axes
for m in qdset["name"].data:
    da, vavg, navg = retrieve_timeseries(m)
    zp = newzp[m]
    Zdust, kwargs = get_kwargs(m)

    cold, unst, warm, whot, hot = get_integrated(zp)

    d, dp1, dp2 = (cold, unst, warm)
    xlist = [
        d["cooling"],
        2 * d["Edot_diss"],
        # dp1['netcool']-dp1['Edot_diss']-dp2['netcool'],
        dp1["netcool"],
        d["Edot_diss"] + dp1["netcool"] - dp1["Edot_diss"],
    ]
    for ax1_, ax2_, x in zip(ax1, ax2, xlist):
        y = d["cooling"] - d["heating"]
        plt.sca(ax1_)
        plot_points(x, y, **kwargs)
        plt.ylabel(r"$L^{\rm u} - G^{\rm u}$")
        plt.sca(ax2_)
        plot_points(x, y / x, **kwargs)
        plt.ylabel(r"$(L^{\rm u} - G^{\rm u})/x$")

for ax in ax1:
    plt.sca(ax)
    plt.plot([1.0e2, 1.0e15], [1.0e2, 1.0e15], color="k", ls="--")
    plt.xlim(1.0e5, 1.0e13)
    plt.ylim(1.0e5, 1.0e13)


for ax, xlab in zip(
    ax2,
    [
        r"$L^{\rm c}$",
        r"$\dot{E}_{\rm diss}^{\rm c}$",
        r"$(L-G)^{\rm u}$",
        # r"$L_{\rm net}^{\rm wh}$",
        r"$\dot{E}_{\rm diss}^{\rm c}+(L-G)^{\rm u}$",
    ],
):
    plt.sca(ax)
    plt.axhline(1, color="k", ls="--")
    plt.xlabel(xlab)
    plt.ylim(0.01, 100)
plt.tight_layout()
plt.savefig("./lowZ-coolheat-figures/netcool_cold_unst.png")

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(15, 4), sharex="col")
# ax1, ax2 = axes
for m in qdset["name"].data:
    da, vavg, navg = retrieve_timeseries(m)
    zp = newzp[m]
    Zdust, kwargs = get_kwargs(m)

    cold, unst, warm, whot, hot = get_integrated(zp)

    for ax, d, dp1 in zip(axes, [whot, warm, unst, cold], [hot, whot, warm, unst]):
        H = np.sqrt(d["rhoz2"] / d["mass"])
        vz = np.sqrt(2.0 * d["Ek3"] / d["mass"])
        tver = H / vz
        Edot = d["Ek"] / tver
        d["tver"] = tver
        d["tcool"] = d["Eth"] / d["netcool"]
        if "tcool" not in dp1:
            dp1["tcool"] = dp1["Eth"] / dp1["netcool"]
        d["Edot_diss"] = Edot
        d["net_cooling"] = d["cooling"] - d["heating"] - d["Edot_diss"]
        if "net_cooling" in dp1:
            d["net_cooling"] -= dp1["net_cooling"]
        plt.sca(ax)

    for ax, d, dp1 in zip(axes, [whot, warm, unst, cold], [hot, whot, warm, unst]):
        plt.sca(ax)
        plot_points(d["tcool"], d["tver"], **kwargs)
        plt.plot([1.0e-12, 1.0e15], [1.0e-12, 1.0e15], color="k", ls="--")
        plt.xlim(1.0e-3, 1.0e3)
        plt.ylim(1.0e-3, 1.0e3)

for ax, d, dp1 in zip(
    axes, ["whot", "warm", "unst", "cold"], ["hot", "whot", "warm", "unst"]
):
    plt.sca(ax)
    plt.xlabel(f"$t_{{\\rm cool}}^{{\\rm {d}}}$")
    plt.ylabel(f"$t_{{\\rm ver}}^{{\\rm {d}}}$")

plt.tight_layout()
plt.savefig(os.path.join(outdir, "tcool_tver_comparison.png"), bbox_inches="tight")

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(15, 4), sharex="col")
# ax1, ax2 = axes
for m in qdset["name"].data:
    da, vavg, navg = retrieve_timeseries(m)
    zp = newzp[m]
    Zdust, kwargs = get_kwargs(m)

    cold, unst, warm, whot, hot = get_integrated(zp)

    for ax, d, dp1 in zip(axes, [whot, warm, unst, cold], [hot, whot, warm, unst]):
        H = np.sqrt(d["rhoz2"] / d["mass"])
        vz = np.sqrt(2.0 * d["Ek3"] / d["mass"])
        tver = H / vz
        Edot = d["Ek"] / tver
        d["tcool"] = d["Eth"] / d["netcool"]
        if "tcool" not in dp1:
            dp1["tcool"] = dp1["Eth"] / dp1["netcool"]
        d["Edot_diss"] = Edot
        d["net_cooling"] = d["cooling"] - d["heating"] - d["Edot_diss"]
        if "net_cooling" in dp1:
            d["net_cooling"] -= dp1["net_cooling"]
        plt.sca(ax)

    for ax, d, dp1 in zip(axes, [whot, warm, unst, cold], [hot, whot, warm, unst]):
        plt.sca(ax)
        plot_points(d["Eth"], dp1["Eth"] / dp1["tcool"] * d["tcool"], **kwargs)
        plt.plot([1.0e5, 1.0e12], [1.0e5, 1.0e12], color="k", ls="--")
        plt.xlim(1.0e5, 1.0e12)
        plt.ylim(1.0e5, 1.0e12)
        # plt.xlim(1.e-3,1.e3)
        # plt.ylim(1.e-3,1.e3)

for ax, d, dp1 in zip(
    axes, ["whot", "warm", "unst", "cold"], ["hot", "whot", "warm", "unst"]
):
    plt.sca(ax)
    plt.xlabel(f"$E_{{\\rm th}}^{{\\rm {d}}}$")
    plt.ylabel(f"$E_{{\\rm th}}^{{\\rm {dp1}}}$")

plt.tight_layout()
plt.savefig(os.path.join(outdir, "Eth_comparison.png"), bbox_inches="tight")